In [1]:
from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import datetime as dt
import json
import psycopg2
from psycopg2 import sql
import json

import datetime
localhost_db_params = {
    "host": "localhost",
    "port": 15432,
    "sslmode": "disable",
    "dbname": "de",
    "user": "jovyan",
    "password": "jovyan"
}

db_params = {
    "host": "rc1a-1kn18k47wuzaks6h.mdb.yandexcloud.net",
    "port": 6432,
    "sslmode": "require",
    "dbname": "de-public",
    "user": "student",
    "password": "student1"
}
import json
from typing import Any, Dict

from bson.objectid import ObjectId


def json2str(obj: Any) -> str:
    return json.dumps(to_dict(obj), sort_keys=True, ensure_ascii=False)


def str2json(str: str) -> Dict:
    return json.loads(str)


def to_dict(obj, classkey=None):
    if isinstance(obj, datetime):
        return obj.strftime("%Y-%m-%d %H:%M:%S")
    elif isinstance(obj, ObjectId):
        return str(obj)
    if isinstance(obj, dict):
        data = {}
        for (k, v) in obj.items():
            data[k] = to_dict(v, classkey)
        return data
    elif hasattr(obj, "_ast"):
        return to_dict(obj._ast())
    elif hasattr(obj, "__iter__") and not isinstance(obj, str):
        return [to_dict(v, classkey) for v in obj]
    elif hasattr(obj, "__dict__"):
        data = dict([(key, to_dict(value, classkey))
                     for key, value in obj.__dict__.items()
                     if not callable(value) and not key.startswith('_')])
        if classkey is not None and hasattr(obj, "__class__"):
            data[classkey] = obj.__class__.__name__
        return data
    else:
        return obj


In [2]:
import psycopg2
from psycopg2 import sql
# import datetime
import json
# Define the database connection parameters
def localhost_insert(db_params, sql, params):
    try:
        conn = psycopg2.connect(**db_params)
        cursor = conn.cursor()
        # insert_query = sql.SQL("INSERT INTO stg.srv_wf_settings (workflow_key, workflow_settings) VALUES (%s, %s) ON CONFLICT (workflow_key) DO UPDATE SET workflow_settings = EXCLUDED.workflow_settings;")
        # cursor.execute(insert_query, ['events_load', workflow_settings])
        cursor.execute(sql, params)
        conn.commit()
        print("Data inserted successfully!")
        cursor.close()
        conn.close()
    
    except (Exception, psycopg2.Error) as error:
        print("Error while inserting data into the database:", error)
def localhost_get(db_params, execute_string):
    try:
        conn = psycopg2.connect(**db_params)
        cursor = conn.cursor()
        cursor.execute(execute_string)
        data = cursor.fetchall()
        cursor.close()
        conn.close()
        return data
    except (Exception, psycopg2.Error) as error:
        print("Error while selecting data:", error)


In [3]:
from pydantic import BaseModel
class UserObj(BaseModel):
    id: int
    user_id: str
    user_name: str
    user_login: str

# USERS

In [ ]:
data = localhost_get(localhost_db_params,  """SELECT id, object_value
                    FROM stg.ordersystem_users
                    WHERE object_id > '0' --Пропускаем те объекты, которые уже загрузили.
                    ORDER BY object_id ASC --Обязательна сортировка по id, т.к. id используем в качестве курсора.
                    LIMIT 10; --Обрабатываем только одну пачку объектов.""")
# for row in data:
#     row = str2json(row[2])
#     data_to_insert = [row['_id'], row['login'], row['name']]
#     sql = "INSERT INTO dds.dm_users (user_id, user_name, user_login) VALUES (%s, %s, %s);"
#     params = [row['_id'], row['login'], row['name']]
#     localhost_insert(localhost_db_params, sql, params)

In [143]:
# s = [str2json(row[2]) for row in data]
new_data = []
for tupple in s:
    id, dict = tupple
    new_item = UserObj(
        id = id,
        user_id= dict['_id'],
        user_name= dict['name'],
        user_login= dict['login']
    )
    new_data.append(new_item)

In [128]:
s = [(row[0],str2json(row[1])) for row in data]

In [112]:
localhost_insert(localhost_db_params, sql, params)

Data inserted successfully!


In [80]:
user = new_data[0]

{'user_id': '626a81ce9a8cd1920641e264',
 'user_name': 'Гаврила Архипович Игнатьев',
 'user_login': 'ninel_2016'}

In [ ]:
sql = """INSERT INTO dds.dm_users(user_id, user_name, user_login)
                    VALUES (%(user_id)s, %(user_name)s, %(user_login)s)
                    ON CONFLICT (user_id) DO UPDATE
                    SET
                        user_name = EXCLUDED.user_name,
                        user_login = EXCLUDED.user_login;
                        """

params = {"user_id": user.user_id,
        "user_name": user.user_name,
        "user_login": user.user_login
         }

# RESTAURANTS

In [152]:
id — автогенерируемое значение, int. Пример: 1.
restaurant_id — оригинальный id ресторана из системы-источника, varchar. Пример: a51e4e31ae4602047ec52534.
restaurant_name — оригинальное имя из системы-источника, varchar. Пример: Кубдари.
active_from — update_ts из системы-источнsика, timestamp. Пример: 2022-10-31 18:00:14.000.
active_to — срок действия записи, timestamp. Если запись актуальна, то 2099-12-31. Пример: 2099-12-31 00:00:00.000.

SyntaxError: invalid character '—' (U+2014) (805207030.py, line 1)

In [251]:
data = localhost_get(localhost_db_params,  """SELECT object_value
                    FROM stg.ordersystem_orders
                    WHERE object_id > '0' --Пропускаем те объекты, которые уже загрузили.
                    ORDER BY object_id ASC --Обязательна сортировка по id, т.к. id используем в качестве курсора.
                    LIMIT 10; --Обрабатываем только одну пачку объектов.""")

In [223]:
class RestaurantObj(BaseModel):
    restaurant_id: str
    restaurant_name: str
    active_from: str
    active_to: str

In [252]:
data[0][0]
s = [str2json(row[0]) for row in data]


In [192]:
s = [str2json(row[0]) for row in data]
new_data = []
for item in s:
    new_item = RestaurantObj(
        restaurant_id = item['_id'],
        restaurant_name = item['name'],
        active_from = item['update_ts'],
        active_to = item['update_ts']
    )
    new_data.append(new_item)

In [193]:
restaurant = new_data[0]

In [210]:
sql = """INSERT INTO dds.dm_restaurants(restaurant_id, restaurant_name, active_from, active_to)
                    VALUES (%(restaurant_id)s, %(restaurant_name)s, %(active_from)s, '2099-12-31 00:00:00.000')
                    ON CONFLICT (restaurant_id) DO UPDATE
                    SET
                    restaurant_id = EXCLUDED.restaurant_id,
                        restaurant_name = EXCLUDED.restaurant_name,
                        active_from = EXCLUDED.active_from,
                        active_to = EXCLUDED.active_to;
                        """

params = {"restaurant_id": restaurant.restaurant_id,
    "restaurant_name": restaurant.restaurant_name,
        "active_from": restaurant.active_from         }

In [211]:
localhost_insert(localhost_db_params, sql, params)

Data inserted successfully!


### TimeStamps

In [4]:
data = localhost_get(localhost_db_params,  """
                    SELECT object_value
                    FROM stg.ordersystem_orders
                    WHERE object_id > '0' --Пропускаем те объекты, которые уже загрузили.
                    and (object_value::json->>'final_status' = 'CLOSED'
                    or object_value::json->>'final_status' = 'CANCELED')
                    ORDER BY object_id ASC --Обязательна сортировка по id, т.к. id используем в качестве курсора.
                    LIMIT 100; --Обрабатываем только одну пачку объектов.
                    """)

In [18]:
class TsObj(BaseModel):
    obj_id: str
    ts: str
    year: int
    month: int
    day: int
    date: datetime.date
    time: datetime.time

In [9]:
data[0][0]
s = [str2json(row[0]) for row in data]


In [13]:
s[0]

'651bab9c407deadbdf80bb4f'

In [19]:
new_data = []
for item in s:
    parsed_datetime = datetime.datetime.strptime(item['date'], '%Y-%m-%d %H:%M:%S')
    formatted_ts = parsed_datetime.strftime('%Y-%m-%d %H:%M:%S.%f')
    date = parsed_datetime.date()
    time = parsed_datetime.time()
    new_item = TsObj(
        obj_id = item['_id'],
        ts = formatted_ts,
        year = parsed_datetime.year,
        month = parsed_datetime.month,
        day = parsed_datetime.day,
        date = date,
        time = time
    )
    new_data.append(new_item)

In [20]:
new_item

TsObj(obj_id='651c2db9a935b6483577e215', ts='2023-10-03 15:05:29.000000', year=2023, month=10, day=3, date=datetime.date(2023, 10, 3), time=datetime.time(15, 5, 29))

In [10]:
ts = new_data[0]

In [21]:
sql = """INSERT INTO dds.dm_timestamps(ts, year, month, day, date, time)
                    VALUES (%(ts)s, %(year)s, %(month)s, %(day)s, %(date)s, %(time)s)
                        """
                    # ON CONFLICT (restaurant_id) DO UPDATE
                    # SET
                    #     restaurant_id = EXCLUDED.restaurant_id,
                    #     restaurant_name = EXCLUDED.restaurant_name,
                    #     active_from = EXCLUDED.active_from,
                    #     active_to = EXCLUDED.active_to;

params = {"ts": ts.ts,
          "year": ts.year,
          "month": ts.month,
          "day": ts.day,
          "date": ts.date,
          "time": ts.time}

NameError: name 'ts' is not defined

In [12]:
localhost_insert(localhost_db_params, sql, params)

Data inserted successfully!


# Products

In [63]:
data = localhost_get(localhost_db_params,  """
                    SELECT object_value
                    FROM stg.ordersystem_restaurants
                    ORDER BY object_id ASC --Обязательна сортировка по id, т.к. id используем в качестве курсора.
                    LIMIT 100; --Обрабатываем только одну пачку объектов.
                    """)

In [49]:
class ProductObj(BaseModel):
    product_id: str
    product_name: str
    product_price: float
    restaurant_id: str
    active_from: str

In [64]:
data
s = [str2json(row[0]) for row in data]

In [102]:
new_data = []
for row in s:
    order_items = row.get('menu', [])
    for item in order_items:
        new_item = ProductObj(
            product_id = item.get('_id', ''),
            product_name = item.get('name', ''),
            product_price = item.get('price', 0),
            restaurant_id = row['_id'],
            active_from = row['update_ts']
        )
        new_data.append(new_item)

In [71]:
new_data[0]

ProductObj(product_id='6276e8cd0cf48b4cded0086c', product_name='ГРИЛАТА ОВОЩНАЯ ПО-МЕКСИКАНСКИ', product_price=60.0, restaurant_id='626a81cfefa404208fe9abae', active_from='2023-11-05 10:00:12')

In [96]:
dates = localhost_get(localhost_db_params,  f"SELECT * FROM dds.dm_restaurants where restaurant_id ='{new_data[42].restaurant_id}' ")

In [97]:
dates

[(7,
  'a51e4e31ae4602047ec52534',
  'Кубдари',
  datetime.datetime(2023, 11, 5, 6, 0, 14),
  datetime.datetime(2099, 12, 31, 0, 0))]

In [99]:
dates[0][1]

'a51e4e31ae4602047ec52534'

In [40]:
new_data[0].product_name

'Лагман'

In [101]:
insert_queries = []
query_template = """INSERT INTO dds.dm_products (product_id, product_name, product_price, active_from, active_to, restaurant_id)
                    VALUES (%(product_id)s, %(product_name)s, %(product_price)s, %(active_from)s, '2099-12-31 00:00:00.000', %(restaurant_id)s)
                """

params = []

for item in new_data:
    restaurant = localhost_get(localhost_db_params,  f"SELECT * FROM dds.dm_restaurants where restaurant_id ='{item.restaurant_id}' and active_to='2099-12-31 00:00:00.000'")
    query_params = {
        "product_id": item.product_id,
        "product_name": item.product_name,
        "product_price": item.product_price,
        "active_from": item.restaurant_id,
        "restaurant_id": restaurant[0][0]
    }

    insert_queries.append(query_template)
    params.append(query_params)

# Print the SQL insert queries and parameters
for query, param in zip(insert_queries, params):
    print(query % param)


INSERT INTO dds.dm_products (product_id, product_name, product_price, active_from, active_to, restaurant_id)
                    VALUES (6276e8cd0cf48b4cded0086c, ГРИЛАТА ОВОЩНАЯ ПО-МЕКСИКАНСКИ, 60.0, 626a81cfefa404208fe9abae, '2099-12-31 00:00:00.000', 4)
                
INSERT INTO dds.dm_products (product_id, product_name, product_price, active_from, active_to, restaurant_id)
                    VALUES (6276e8cd0cf48b4cded0086d, ГРИЛАТА С КУРИЦЕЙ И ОВОЩАМИ, 60.0, 626a81cfefa404208fe9abae, '2099-12-31 00:00:00.000', 4)
                
INSERT INTO dds.dm_products (product_id, product_name, product_price, active_from, active_to, restaurant_id)
                    VALUES (6276e8cd0cf48b4cded0086e, РОЛЛ С ВЕТЧИНОЙ И ОМЛЕТОМ, 120.0, 626a81cfefa404208fe9abae, '2099-12-31 00:00:00.000', 4)
                
INSERT INTO dds.dm_products (product_id, product_name, product_price, active_from, active_to, restaurant_id)
                    VALUES (6276e8cd0cf48b4cded0086f, РОЛЛ С КУРИЦЕЙ И ПЕСТО

In [ ]:
localhost_insert(localhost_db_params, sql, params)

# Orders

In [140]:
class OrderObj(BaseModel):
    key: str
    status: str
    restaurant_long_id: str
    ts_long_id: str
    user_long_id: str

In [157]:
data = localhost_get(localhost_db_params,  """
                    SELECT object_value
                    FROM stg.ordersystem_orders
                    ORDER BY object_id ASC --Обязательна сортировка по id, т.к. id используем в качестве курсора.
                    LIMIT 10000; --Обрабатываем только одну пачку объектов.
                    """)

# users =     restaurant = localhost_get(localhost_db_params,  f"SELECT * FROM dds.dm_restaurants where restaurant_id ='{item.restaurant_id}' and active_to='2099-12-31 00:00:00.000'")

s = [str2json(row[0]) for row in data]
new_data = []
for item in s:
    new_item = OrderObj(
        key = item.get('_id', ''),
        status = item.get('final_status', ''),
        restaurant_long_id = item.get('restaurant').get('id'),
        ts_long_id = item.get('update_ts'),
        user_long_id = item.get('user').get('id')
    )
    new_data.append(new_item)

In [145]:
new_data[0]

OrderObj(key='651bab9c407deadbdf80bb4f', status='CLOSED', restaurant_long_id='ebfa4c9b8dadfc1da37ab58d', ts_long_id='2023-10-03 05:50:20', user_long_id='626a81ce9a8cd1920641e295')

In [155]:
    ts = localhost_get(localhost_db_params,  f"SELECT * FROM dds.dm_timestamps where ts ='{item.ts_long_id}' ")
ts[0][0]

53147

In [159]:
insert_queries = []
query_template = """INSERT INTO dds.dm_orders (order_key, order_status, restaurant_id, timestamp_id, user_id)
                    VALUES (%(order_key)s, %(order_status)s, %(restaurant_id)s, %(timestamp_id)s, %(user_id)s)
                    ON CONFLICT (order_key) DO UPDATE
                                SET order_status = EXCLUDED.order_status, 
                                    restaurant_id = EXCLUDED.restaurant_id, 
                                    timestamp_id = EXCLUDED.timestamp_id, 
                                    user_id = EXCLUDED.user_id;
                """

params = []




for item in new_data:
    restaurant = localhost_get(localhost_db_params,  f"SELECT * FROM dds.dm_restaurants where restaurant_id ='{item.restaurant_long_id}' and active_to='2099-12-31 00:00:00.000'")
    user = localhost_get(localhost_db_params,  f"SELECT * FROM dds.dm_users where user_id ='{item.user_long_id}'")
    ts = localhost_get(localhost_db_params,  f"SELECT * FROM dds.dm_timestamps where ts ='{item.ts_long_id}' ")

    print(ts, item.ts_long_id)
    query_params = {
        "order_key": item.key,
        "order_status": item.status,
        "restaurant_id": restaurant[0][0],
        "timestamp_id": ts[0][0],
        "user_id": user[0][0]
    }

    insert_queries.append(query_template)
    params.append(query_params)

# # Print the SQL insert queries and parameters
# for query, param in zip(insert_queries, params):
#     localhost_insert(localhost_db_params, query, param)
#     print(query % param)


[(53048, datetime.datetime(2023, 10, 3, 5, 50, 20), 2023, 10, 3, datetime.time(5, 50, 20), datetime.date(2023, 10, 3))] 2023-10-03 05:50:20
[(53049, datetime.datetime(2023, 10, 3, 5, 55, 17), 2023, 10, 3, datetime.time(5, 55, 17), datetime.date(2023, 10, 3))] 2023-10-03 05:55:17
[(53050, datetime.datetime(2023, 10, 3, 6, 0, 25), 2023, 10, 3, datetime.time(6, 0, 25), datetime.date(2023, 10, 3))] 2023-10-03 06:00:25
[(53051, datetime.datetime(2023, 10, 3, 6, 5, 18), 2023, 10, 3, datetime.time(6, 5, 18), datetime.date(2023, 10, 3))] 2023-10-03 06:05:18
[(53052, datetime.datetime(2023, 10, 3, 6, 10, 19), 2023, 10, 3, datetime.time(6, 10, 19), datetime.date(2023, 10, 3))] 2023-10-03 06:10:19
[(53053, datetime.datetime(2023, 10, 3, 6, 15, 19), 2023, 10, 3, datetime.time(6, 15, 19), datetime.date(2023, 10, 3))] 2023-10-03 06:15:19
[(53054, datetime.datetime(2023, 10, 3, 6, 20, 20), 2023, 10, 3, datetime.time(6, 20, 20), datetime.date(2023, 10, 3))] 2023-10-03 06:20:20
[(53055, datetime.dateti

IndexError: list index out of range

# FCT

In [162]:
data = localhost_get(localhost_db_params, """
select
    s.rn,
    p.id as product_id,
    o.id as order_id,
    s.quantity::int as count,
    s.price::numeric(19, 5) as price,
    s.quantity::int * s.price::numeric(19, 5) as total_sum,
    coalesce(s.bonus_payment, '0')::numeric(19,2) as bonus_payment,
    coalesce(s.bonus_grant, '0')::numeric(19,2) as bonus_grant
from (
    select 
        ROW_NUMBER() OVER() as rn,
        o.order_key,
        o.product_key,
        o.quantity,
        o.price,
        a.bonus_grant,
        a.bonus_payment
    from 
    (select
        object_id as order_key,
        json_array_elements((object_value::JSON->>'order_items')::JSON)::JSON->>'id' as product_key,
        json_array_elements((object_value::JSON->>'order_items')::JSON)::JSON->>'quantity' as quantity,
        json_array_elements((object_value::JSON->>'order_items')::JSON)::JSON->>'price' as price
    from stg.ordersystem_orders where (object_value::JSON->>'final_status')::varchar in ('CLOSED')) o
    left join (select 
        (be.event_value::JSON->>'order_id')::varchar as order_key,
        json_array_elements((event_value::JSON->>'product_payments')::JSON)::JSON->>'product_id' as product_key,
        json_array_elements((event_value::JSON->>'product_payments')::JSON)::JSON->>'bonus_payment' as bonus_payment,
        json_array_elements((event_value::JSON->>'product_payments')::JSON)::JSON->>'bonus_grant' as bonus_grant
    from stg.bonussystem_events be 
    where be.event_type = 'bonus_transaction') a on o.order_key = a.order_key and o.product_key = a.product_key
    ) s
    join dds.dm_orders o on s.order_key = o.order_key
    join dds.dm_products p on s.product_key = p.product_id""")

In [205]:
import decimal
class FctObj(BaseModel):
    product_id: int
    order_id: int
    count: int
    price: decimal.Decimal
    total_sum:  decimal.Decimal
    bonus_payment:  decimal.Decimal
    bonus_grant:  decimal.Decimal

In [199]:
type(data[6][0])

int

In [206]:
# s = [str2json(row[0]) for row in data]
new_data = []
for item in data:
    new_item = FctObj(
        product_id = item[1],
        order_id = item[2],
        count = item[3],
        price = item[4],
        total_sum = item[5],
        bonus_payment = item[6],
        bonus_grant = item[7]
    )
    new_data.append(new_item)

In [208]:
new_data[0]

FctObj(product_id=77, order_id=13240, count=2, price=Decimal('350.00000'), total_sum=Decimal('700.00000'), bonus_payment=Decimal('204.00'), bonus_grant=Decimal('25.00'))